In [3]:
# Dependencies
from census import Census
from API_keys import gkey
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states


# Census API Key
c = gkey

In [ ]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
#census_data = c.acs5.get(("B01003_001E", "B23025_005E"), {
                         #'for': 'zip code tabulation area:*'})

# Convert to DataFrame
#census_pd = pd.DataFrame(census_data)

# Column Reordering
#census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      #"B23025_005E": "Unemployment Count",
                                      #"zip code tabulation area": "Zipcode"})

# Add in Employment Rate (Employment Count / Population)
#census_pd["Unemployment Rate"] = 100 * \
#    census_pd["Unemployment Count"].astype(
#        int) / census_pd["Population"].astype(int)

# Final DataFrame
#census_pd = census_pd[["Zipcode", "Population", "Unemployment Rate"]]

# Visualize
#print(len(census_pd))
#census_pd.head()